In [1]:
import json

with open("./params.json", mode = "r", encoding = "utf-8") as f:
    data = json.load(f)
    seed_val = data["seed_val"]
    model_path = data["model_path"]
    dataset_path_train = data["dataset_path"]["train"]
    dataset_path_val = data["dataset_path"]["validation"]
    dataset_path_test = data["dataset_path"]["test"]
    num_single_sample_timesteps = data["num_single_sample_timesteps"]
    window_stride = data["window_stride"]
    input_window_length = data["input_window_length"]
    label_window_length = data["label_window_length"]
    input_features = data["input_features"]
    label_features = data["label_features"]
    positional_encoding_max_len = data["positional_encoding_max_len"]
    embedding_dim = data["embedding_dim"]
    num_attention_head = data["num_attention_head"]
    num_encoder_layers = data["num_encoder_layers"]
    num_decoder_layers = data["num_decoder_layers"]
    position_wise_nn_dim = data["position_wise_nn_dim"]
    dropout = data["dropout"]
    batch_size = data["batch_size"]
    epochs = data["epochs"]
    learning_rate = data["learning_rate"]

In [2]:
import random
import numpy as np
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(seed_val)
random.seed(seed_val)
np.random.seed(seed_val)

In [3]:
from torch.utils.data import DataLoader
from torchmetrics.regression import R2Score

from utils.pipeline.Data import get_mean_std_respected_temporal, WindowedIterableDataset
from utils.pipeline.Model import TimeSeriesHuggingFaceTransformer
from utils.pipeline.Run import train, validate
from utils.pipeline.Monitor import Overfit

/users/labnet5/gr5/abahari/Documents/Thesis/src/thesis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
##### TRAIN #####
# stats = get_mean_std(
#     dataset_path = dataset_path_train,
#     cols = input_features
# )

stats = get_mean_std_respected_temporal(
    dataset_path = dataset_path_train,
    cols = input_features,
    num_single_sample_timesteps = num_single_sample_timesteps,
    input_window_len = input_window_length,
    label_window_len = label_window_length,
    window_stride = window_stride
)

df_train = WindowedIterableDataset(
    dataset_path = dataset_path_train,
    stats = stats,
    input_features = input_features,
    label_features = label_features,
    num_single_sample_timesteps = num_single_sample_timesteps,
    stride = window_stride,
    input_window_length = input_window_length,
    label_window_length = label_window_length
)

data_loader_train = DataLoader(
    df_train,
    batch_size = batch_size,
    # num_workers = 0,
    # prefetch_factor = 12,
    # persistent_workers = False,
    pin_memory = True
)



##### VALIDATION #####
df_val = WindowedIterableDataset(
    dataset_path = dataset_path_val,
    stats = stats,
    input_features = input_features,
    label_features = label_features,
    num_single_sample_timesteps = num_single_sample_timesteps,
    stride = window_stride,
    input_window_length = input_window_length,
    label_window_length = label_window_length
)

data_loader_val = DataLoader(
    df_val,
    batch_size = batch_size,
    # num_workers = 0,
    # prefetch_factor = 12,
    # persistent_workers = False,
    pin_memory = True
)



##### TEST #####
df_test = WindowedIterableDataset(
    dataset_path = dataset_path_test,
    stats = stats,
    input_features = input_features,
    label_features = label_features,
    num_single_sample_timesteps = num_single_sample_timesteps,
    stride = window_stride,
    input_window_length = input_window_length,
    label_window_length = label_window_length
)

data_loader_test = DataLoader(
    df_test,
    batch_size = batch_size,
    # num_workers = 0,
    # prefetch_factor = 12,
    # persistent_workers = False,
    pin_memory = True
)

In [5]:
model = TimeSeriesHuggingFaceTransformer(
    input_dim = len(input_features),
    output_dim = len(label_features),
    d_model = embedding_dim,
    num_head = num_attention_head,
    num_encoder_layers = num_encoder_layers,
    num_decoder_layers = num_encoder_layers,
    position_wise_ffn_dim = position_wise_nn_dim,
    dropout = dropout
).to(device)

overfit_monitor = Overfit()

print(f"Number of trainable parameters in the model: {sum(p.numel() for p in model.parameters() if p.requires_grad)}\n")

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(),
    lr = learning_rate
)

train_r2 = R2Score(multioutput = "uniform_average").to(device)
val_r2 = R2Score(multioutput = "uniform_average").to(device)

train_per_timestep_r2 = [R2Score(multioutput = "uniform_average").to(device) for _ in range(label_window_length)]
val_per_timestep_r2 = [R2Score(multioutput = "uniform_average").to(device) for _ in range(label_window_length)]

overfit_count = 0

for epoch in range(epochs):
    train_loss, _ = train(
        model = model,
        optimizer = optimizer,
        criterion = criterion,
        r2 = train_r2,
        per_timestep_r2 = train_per_timestep_r2,
        data_loader = data_loader_train,
        device = device,
        epoch = epoch,
        total_epochs = epochs
    )

    val_loss, _ = validate(
        model = model,
        criterion = criterion,
        r2 = val_r2,
        per_timestep_r2 = val_per_timestep_r2,
        data_loader = data_loader_val,
        device = device,
        epoch = epoch,
        total_epochs = epochs
    )

    if(overfit_monitor.check(epoch = epoch, train_loss = train_loss, val_loss = val_loss)):
        break

Number of trainable parameters in the model: 1187334



Epoch 1/15: 717it [00:56, 12.61it/s, train_loss=0.202197]


Epoch [1/15], Train Loss: 0.255438, Train R2: 0.762548

Worst 5 Time-Steps Train R2:
    Time-step 1: R2 = 0.601804
    Time-step 2: R2 = 0.745500
    Time-step 3: R2 = 0.754831
    Time-step 4: R2 = 0.759542
    Time-step 50: R2 = 0.762279
Best 5 Time-Steps Train R2:
    Time-step 14: R2 = 0.769475
    Time-step 18: R2 = 0.769312
    Time-step 17: R2 = 0.769219
    Time-step 12: R2 = 0.768972
    Time-step 15: R2 = 0.768956



Epoch 1/15: 207it [00:32,  6.45it/s, val_loss=0.591205]


Epoch [1/15], Val Loss: 0.876918, Val R2: 0.280655

Worst 5 Time-Steps Val R2:
    Time-step 50: R2 = -0.185781
    Time-step 49: R2 = -0.180037
    Time-step 48: R2 = -0.173331
    Time-step 47: R2 = -0.166245
    Time-step 46: R2 = -0.158242
Best 5 Time-Steps Val R2:
    Time-step 1: R2 = 0.797471
    Time-step 2: R2 = 0.786942
    Time-step 3: R2 = 0.774830
    Time-step 4: R2 = 0.761241
    Time-step 5: R2 = 0.747236

-----------------------------------------------------------------



...

Epoch [12/15], Val Loss: 0.585306, Val R2: 0.512427

Worst 5 Time-Steps Val R2:
    Time-step 50: R2 = 0.161595
    Time-step 49: R2 = 0.175311
    Time-step 48: R2 = 0.188031
    Time-step 47: R2 = 0.202074
    Time-step 46: R2 = 0.216824
Best 5 Time-Steps Val R2:
    Time-step 1: R2 = 0.818471
    Time-step 2: R2 = 0.809687
    Time-step 3: R2 = 0.800769
    Time-step 4: R2 = 0.791602
    Time-step 5: R2 = 0.782013

-----------------------------------------------------------------

Possible Overfitting!!! 2/3



Epoch 13/15: 717it [00:59, 11.98it/s, train_loss=0.190089]


Epoch [13/15], Train Loss: 0.185274, Train R2: 0.818402

Worst 5 Time-Steps Train R2:
    Time-step 1: R2 = 0.798734
    Time-step 2: R2 = 0.813912
    Time-step 3: R2 = 0.816766
    Time-step 4: R2 = 0.817978
    Time-step 5: R2 = 0.818364
Best 5 Time-Steps Train R2:
    Time-step 22: R2 = 0.819107
    Time-step 31: R2 = 0.819105
    Time-step 21: R2 = 0.819100
    Time-step 34: R2 = 0.819096
    Time-step 24: R2 = 0.819072



Epoch 13/15: 207it [00:32,  6.43it/s, val_loss=0.500393]

Epoch [13/15], Val Loss: 0.596990, Val R2: 0.505712

Worst 5 Time-Steps Val R2:
    Time-step 50: R2 = 0.153202
    Time-step 49: R2 = 0.165724
    Time-step 48: R2 = 0.177931
    Time-step 47: R2 = 0.191003
    Time-step 46: R2 = 0.204948
Best 5 Time-Steps Val R2:
    Time-step 1: R2 = 0.818966
    Time-step 2: R2 = 0.810412
    Time-step 3: R2 = 0.801852
    Time-step 4: R2 = 0.793139
    Time-step 5: R2 = 0.784156

-----------------------------------------------------------------

Possible Overfitting!!! 3/3

Training Stopped!!!


In [6]:
torch.save(model, model_path)